# Project 1  
### The Moonshadow Elves  
* Hongmei Zeng  
* Noah Stettler  
* Patrick Harrington  
* Yacub Bholat  
  
Due: 13 January 2020  

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import requests

# HONGMEI CODE

In [ ]:
path = os.path.join('Data', 'SeriesReport-20200112162716_5da8f4.csv')
df = pd.read_csv(path)
df.head(20)

In [ ]:
df['Unnamed: 1'][2]

In [ ]:
cleaned_df = df.drop([0,1,2,3,4,5,6,7,8,9])
cleaned_df.head()

In [ ]:
names = {'Local Area Unemployment Statistics': 'Year',
         'Unnamed: 1': 'Month',
         'Unnamed: 2': 'Labor Force',
         'Unnamed: 3': 'Employment',
         'Unnamed: 4': 'Unemployment',
         'Unnamed: 5': 'Unemployment Rate %' }
cleaned_df = cleaned_df.rename(columns=names)
cleaned_df = cleaned_df.reset_index(drop=True)
cleaned_df.tail(10)

In [ ]:
cleaned_df['Labor Force'] = cleaned_df['Labor Force'].astype('int')
cleaned_df['Employment'] = cleaned_df['Employment'].astype('int')
cleaned_df['Unemployment'] = cleaned_df['Unemployment'].astype('int')
cleaned_df['Unemployment Rate %'] = cleaned_df['Unemployment Rate %'].astype('float')

In [ ]:
cleaned_df['Employment Rate %'] = ((cleaned_df['Employment'] / cleaned_df['Labor Force']) * 100).round(1) 
cleaned_df.head(10)

In [ ]:
new_df = cleaned_df[['Year','Month','Labor Force','Employment','Employment Rate %','Unemployment','Unemployment Rate %']]
new_df.head(20)

In [ ]:
# Delete the Data of the year 2019
new_df = new_df[new_df.Year != '2019']
new_df

In [ ]:
yearly_df = new_df[new_df.Month=='Annual'].drop(columns=['Month'])
yearly_df.set_index(['Year'], inplace = True)
yearly_df

In [ ]:
x = yearly_df.index
y = yearly_df["Employment"]
plt.plot(x, y,color='blue', marker='o', linewidth=2, markersize=8)

plt.xlabel('Year')
plt.ylabel('Employment Amount')
plt.title('Employment Amount of Austin City(2010-2018)', fontsize=12)
plt.grid(axis='both', linestyle='-')
plt.savefig('Fig1 Employment Amount.png')
plt.show()

In [ ]:
x = yearly_df.index
y = yearly_df['Employment Rate %']
plt.plot(x, y,color='green', marker='o', linewidth=2, markersize=8)

plt.xlabel('Year')
plt.ylabel('Employment Rate (%)')
plt.title('Employment Rate of Austin City(2010-2018)', fontsize=12)
plt.ylim(80, max(y) + 10)
plt.grid(axis='both', linestyle='-')
plt.savefig('Fig2 Employment Rate.png')
plt.show()

In [ ]:
monthly_df = new_df[new_df.Month!='Annual']
monthly_df

In [ ]:
monthly_employment_df = pd.pivot_table(monthly_df, 
                            index='Year',
                            columns='Month', 
                            values='Employment')
monthly_employment_df = monthly_employment_df[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']]
monthly_employment_df

In [ ]:
colors = ['red', 'blue', 'green', 'orange','cyan', 'magenta', 'yellow', 'black', 'grey','brown','silver', 'purple']
markers = ['^', 'o', 's', '*', '8', 'p', 'X', 'D', 'x', 'h', '+', '|']

x = monthly_employment_df.index 
for column, i in zip(monthly_employment_df.columns, range(len(monthly_employment_df.columns))):
    y = monthly_employment_df[column]
    plt.errorbar(x, y, yerr=3, elinewidth=0.8, 
                 color=colors[i], marker=markers[i], 
                 linestyle=':', linewidth=1,
                 errorevery=1, label=column)
    
plt.xlabel('Year')
plt.ylabel('Employment')
plt.title('Monthly Employment Amount(2010-2018)', fontsize=12)
plt.legend(loc='lower right')
plt.ylim(400000, max(y) + 10000)
plt.grid(axis='y', linestyle='-')
plt.figure(figsize=(15,9))
plt.savefig('Fig3 Monthly Employment Amount.png')
plt.show()

In [ ]:
monthly_employment_rate_df = pd.pivot_table(monthly_df, 
                            index='Year',
                            columns='Month', 
                            values='Employment Rate %')
monthly_employment_rate_df = monthly_employment_rate_df[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']]
monthly_employment_rate_df

In [ ]:
colors = ['red', 'blue', 'green', 'orange','cyan', 'magenta', 'yellow', 'black', 'grey','brown','silver', 'purple']
markers = ['^', 'o', 's', '*', '8', 'p', 'X', 'D', 'x', 'h', '+', ',']

x = monthly_employment_rate_df.index 
for column, i in zip(monthly_employment_rate_df.columns, range(len(monthly_employment_rate_df.columns))):
    y = monthly_employment_rate_df[column]
    plt.errorbar(x, y, yerr=0.5, elinewidth=0.8, 
                 color=colors[i], marker=markers[i], 
                 linestyle=':', linewidth=1,
                 errorevery=1, label=column)
    
plt.xlabel('Year')
plt.ylabel('Employment Rate %')
plt.title('Monthly Employment Rate(2010-2018)', fontsize=12)
plt.legend(loc='lower right')
plt.ylim(90, max(y) + 2.5)
plt.grid(axis='y', linestyle='-')
plt.figure(figsize=(15,9))
plt.savefig('Fig4 Monthly Employment Rate.png')
plt.show()

In [ ]:
monthly_df

In [ ]:
monthly_df['Year-Month'] = monthly_df['Year'] + '-' + monthly_df['Month']
monthly_df

In [ ]:
x = monthly_df['Year-Month']
y = monthly_df["Employment"]
plt.plot(x, y,color='red', marker='o', linewidth=2, markersize=8)

plt.xlabel('Year-Month')
plt.ylabel('Employment Amount')
plt.title('Employment Statistics of Austin City(2010-2018)', fontsize=12)
plt.grid(axis='both', linestyle='-')
plt.figure(figsize=(15,9))
plt.savefig('Fig5 Employment Statistics.png')
plt.show()

In [ ]:
def df(year):
    filename = 'All Industries Employment ' + str(year) + '.csv'
    path = os.path.join('Data', filename)
    df = pd.read_csv(path)
    df

In [ ]:
path2010 = os.path.join('Data', 'All Industries Employment 2010.csv')
df_2010 = pd.read_csv(path2010)

df_2010 = df_2010[df_2010.Area=='Travis County, Texas']
df_2010 = df_2010[df_2010.Industry != 'Total, all industries']
df_2010 = df_2010[['Industry','Annual Average Employment']]
cleaned_df_2010 = df_2010.reset_index(drop=True)
cleaned_df_2010

In [ ]:
path2010 = os.path.join('Data', 'All Industries Employment 2010.csv')
df_2010 = pd.read_csv(path2010)
df_2010

path2011 = os.path.join('Data', 'All Industries Employment 2011.csv')
df_2011 = pd.read_csv(path2011)
df_2011

path2012 = os.path.join('Data', 'All Industries Employment 2012.csv')
df_2012 = pd.read_csv(path2012)
df_2012

path2013 = os.path.join('Data', 'All Industries Employment 2013.csv')
df_2013 = pd.read_csv(path2013)
df_2013

path2014 = os.path.join('Data', 'All Industries Employment 2014.csv')
df_2014 = pd.read_csv(path2014)
df_2014

path2015 = os.path.join('Data', 'All Industries Employment 2015.csv')
df_2015 = pd.read_csv(path2015)
df_2015

path2016 = os.path.join('Data', 'All Industries Employment 2016.csv')
df_2016 = pd.read_csv(path2016)
df_2016

path2017 = os.path.join('Data', 'All Industries Employment 2017.csv')
df_2017 = pd.read_csv(path2017)
df_2017

path2018 = os.path.join('Data', 'All Industries Employment 2018.csv')
df_2018 = pd.read_csv(path2018)
df_2018
